In [ ]:
import os
import torch
import tqdm
import math
import gpytorch
from torch.nn import Linear
from gpytorch.means import ConstantMean, LinearMean
from gpytorch.kernels import MaternKernel, ScaleKernel
from gpytorch.variational import VariationalStrategy, CholeskyVariationalDistribution, \
    LMCVariationalStrategy
from gpytorch.distributions import MultivariateNormal
from gpytorch.models.deep_gps import DeepGPLayer, DeepGP
from gpytorch.mlls import DeepApproximateMLL, VariationalELBO
from gpytorch.likelihoods import MultitaskGaussianLikelihood

In [ ]:
from matplotlib import pyplot as plt

smoke_test = ('CI' in os.environ)
%matplotlib inline

In [1]:
import torch
import gpytorch
from gpytorch.kernels import SpectralMixtureKernel
from gpytorch.means import ConstantMean
from gpytorch.distributions import MultivariateNormal
from gpytorch.models import IndependentModelList, ExactGP
from gpytorch.likelihoods import MultitaskGaussianLikelihood

# Create synthetic data
train_x = torch.linspace(0, 1, 100)
train_y1 = torch.sin(train_x * (2 * 3.1415)) + 0.1 * torch.randn_like(train_x)
train_y2 = torch.cos(train_x * (2 * 3.1415)) + 0.1 * torch.randn_like(train_x)

train_y = torch.stack([train_y1, train_y2], dim=-1)

# Define a multi-output GP model with Spectral Mixture Kernel
class MultiOutputGP(ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultiOutputGP, self).__init__(train_x, train_y, likelihood)
        
        # Define mean function
        self.mean = ConstantMean()
        
        # Define kernel
        self.covar_module = SpectralMixtureKernel(num_mixtures=4)
        
    def forward(self, x):
        mean_x = self.mean(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

# Initialize likelihood and model
likelihood = MultitaskGaussianLikelihood(num_tasks=2)
model = MultiOutputGP(train_x, train_y, likelihood)

# Training the model
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# Training loop
n_iter = 50
for i in range(n_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    
    # Output from model
    output = model(train_x)
    
    # Calculate loss and backpropagate
    loss = -mll(output, train_y)
    loss.backward()
    
    print(f"Iteration {i+1}/{n_iter} - Loss: {loss.item()}")
    
    optimizer.step()

# Set model and likelihood to evaluation mode
model.eval()
likelihood.eval()

# Test the model on some test data
test_x = torch.linspace(0, 1, 100)
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    test_y_pred = model(test_x)

# Get the predicted means and covariances
mean_predictions = test_y_pred.mean
covar_predictions = test_y_pred.covariance_matrix

# You can use these mean and covariance predictions as needed.

AttributeError: 'MultivariateNormal' object has no attribute '_interleaved'